In [ ]:
class Node:
  def __init__(self,key,value):
    self.key = key
    self.value = value
    self.next = None

In [ ]:
class HashTableChain:
    def __init__(self, size, file):
        self.size = size
        self.table = [None] * size
        self.g = 31  # positive constant
        with open(file,'r') as f:
          for line in f:
            self.insert(line.strip(),1)
        with open('Chaining.txt','w') as f:
          for i in range(self.size):
            temp = self.table[i]
            while temp!=None:
              f.write(f'{temp.key}: {temp.value}\n')
              temp = temp.next

    # hash function to determine the index for a given key
    def hash_function(self, key):
        hash = 0
        n = len(key)
        for i in range(n):
          hash = (self.g * hash) + ord(key[i])
        index = hash % self.size
        return index

    # insert a key-value pair to the hash table
    def insert(self, key, value):
        hash_val = self.hash_function(key)
        if(self.table[hash_val] == None):
          node = Node(key,value)
          self.table[hash_val] = node
        else:
          temp = self.table[hash_val]
          while temp.key != key and temp.next != None:
            temp = temp.next
          if(temp.key != key and temp.next == None):
            node = Node(key,value)
            temp.next = node
          else:
            temp.value = temp.value + value

    # retrieve the value for a given key
    def search(self, key):
        index = self.hash_function(key)
        val = None
        myNode = self.table[index]
        while myNode != None:
          if(myNode.key == key):
            val = myNode.value
            break
          myNode = myNode.next
        return val

    def delete(self, key):
        hash_val = self.hash_function(key)
        temp1 = self.table[hash_val]
        temp2 = None
        while temp1 != None:
          if(temp1.key == key):
            if(temp2 == None):
              self.table[hash_val] = temp1.next
            else:
              temp2.next = temp1.next
            return temp1.value
          temp2 = temp1
          temp1 = temp1.next
        return None

Strategy Used - **Chaining**

Primary Data Structure is **Hash Table** represented as a list(self.table) of a specified size(self.size), here i am using 58111.(check at the end of file)

Each element in self.table stores reference to a **LinkedList**.

When a collision occurs, we append the data to the end of linkedlist corresponding to index in hash table if the key doesn't exist already with value = 1, else we find the node with the given key at corresponding index then we increament the value by 1.

The **value** tells about the **frequency** about the **key**

In [ ]:
class HashTableLinProb:
    def __init__(self, size, file):
        self.size = size
        self.table = [None] * size
        self.g = 31  # positive constant
        with open(file,'r') as f:
          for line in f:
            self.insert(line.strip(),1)
        with open('Probing.txt','w') as f:
          for i in range(self.size):
            temp = self.table[i]
            if(temp!=None and temp[0]!='---Deleted Node---'):
              f.write(f'{temp[0]}: {temp[1]}\n')

    # hash function to determine the index for a given key
    def hash_function(self, key):
        hash = 0
        n = len(key)
        for i in range(n):
          hash = (self.g * hash) + ord(key[i])
        index = hash % self.size
        return index

    # insert a key-value pair to the hash table
    def insert(self, key, value):
        hash_val = self.hash_function(key)
        temp = self.table[hash_val]
        while temp != None:
          if(temp[0] == key):
            break
          hash_val = (hash_val + 1) % self.size
          temp = self.table[hash_val]
        if(temp == None):
          self.table[hash_val] = (key,value)
        else:
          self.table[hash_val] = (key,self.table[hash_val][1]+1)

    # retrieve the value for a given key
    def search(self, key):
        index = self.hash_function(key)
        val = None
        count = 0
        while count < self.size:
          if(self.table[index][0] == key):
            val = self.table[index][1]
            break
          index = (index+1)%self.size
          count+=1
        return val

    def delete(self, key):
        index = self.hash_function(key)
        val = None
        count = 0
        while count < self.size:
          if(self.table[index][0] == key):
            val = self.table[index][1]
            self.table[index] = ('---Deleted Node---',1)
            break
          index = (index+1)%self.size
          count+=1
        return val

Strategy Used - **Linear Probing**

Data Structure used is **Array**(List in Python). Here Im using of size 58111.(check end of file) Each element in List is a **tuple (key, value)**, where **value** tells about **frequency** of key

When a collision occurs, we check if collision is of same key if yes we increment the value, else we go to next slot and repeat the same process but if we get an empty slot then we insert the tuple (key,1) as key didn't present earlier.

One important thing I did in deletion is that I am not making that slot available to insert once its deleted as it causes the following problem:

Suppose word1 and word2 produces same hash value, so i got word1 for first time its hash value by k, suppose k in hash table be None so I inserted (word1,1) at k, now when word2 is inserted it would get inserted at next availble slot after k as k is already full and let it be l with (word2,1) now if I delete word1, slot k becomes empty and after that if I try to insert word2 it goes to slot k as it is empty and it will cause duplicates for that reason when ever a delete is performed im replacing slot with ('---Deleted Node---',1), so nothing gets inserted there.

In [ ]:
class HashTableDoubleHash:
    def __init__(self, size, file):
        self.size = size
        self.table = [None] * size
        self.g = 31  # positive constant
        with open(file,'r') as f:
          for line in f:
            self.insert(line.strip(),1)
        with open('DoubleHashing.txt','w') as f:
          for i in range(self.size):
            temp = self.table[i]
            if(temp!=None and temp[0]!='---Deleted Node---'):
              f.write(f'{temp[0]}: {temp[1]}\n')

    # hash function to determine the index for a given key
    def hash_function(self, key):
        hash = 0
        n = len(key)
        for i in range(n):
          hash = (self.g * hash) + ord(key[i])
        index = hash % self.size
        return index

    def secondary_hash_function(self, key):
        return (7*key+11) % self.size

    # insert a key-value pair to the hash table
    def insert(self, key, value):
        hash_val = self.hash_function(key)
        temp = self.table[hash_val]
        newHash = [hash_val]
        while(temp!=None):
          if(temp[0] == key):
            break
          hash_val = (self.secondary_hash_function(hash_val))%self.size
          temp = self.table[hash_val]
        if(temp == None):
          self.table[hash_val] = (key,value)
        else:
          self.table[hash_val] = (key, self.table[hash_val][1]+1)

    # retrieve the value for a given key
    def search(self, key):
        hash_val = self.hash_function(key)
        val = None
        count = 0
        while count < self.size:
          if(self.table[hash_val][0] == key):
            val = self.table[hash_val][1]
            break
          hash_val = (self.secondary_hash_function(hash_val))%self.size
          count+=1
        return val

    def delete(self, key):
        hash_val = self.hash_function(key)
        val = None
        count = 0
        while count < self.size:
          if(self.table[hash_val][0] == key):
            val = self.table[hash_val][1]
            self.table[hash_val] = ('---Deleted Node---',1)
            break
          hash_val = (self.secondary_hash_function(hash_val))%self.size
          count+=1
        return val

Strategy Used - **Double Hashing**

Data Structure used is **Array**(List in Python). Here Im using of size 158111.(check end of file) Each element in List is a **tuple (key, value)**, where **value** tells about **frequency** of key

When a collision occurs, we check if collision is of same key if yes we increment the value, else we apply second hash and see if its None then insert else if in collision if key is same then increment the value else repeatedly apply secondary hash function.

Similar to Linear Probing, I did in deletion is that I am not making that slot available to insert once its deleted as it causes the following problem:

Suppose word1 and word2 produces same hash value, so i got word1 for first time its hash value by k, suppose k in hash table be None so I inserted (word1,1) at k, now when word2 is inserted it would get inserted at next availble slot after k as k is already full and let it be l with (word2,1) now if I delete word1, slot k becomes empty and after that if I try to insert word2 it goes to slot k as it is empty and it will cause duplicates for that reason when ever a delete is performed im replacing slot with ('---Deleted Node---',1), so nothing gets inserted there.

## Here I have given sizes for 3 hashTables and initialized all 3 objects so 3 files "Chaining.txt", "Probing.txt" and "DoubleHashing.txt" files for frequency from "dictionary.txt" are generated.

In [ ]:
if __name__ == "__main__":
  HashTableChain(58111,'dictionary.txt')
  HashTableLinProb(58111,'dictionary.txt')
  HashTableDoubleHash(158111,'dictionary.txt')

### See as I already know there are 58110 unique words(calculated separately) in dictionary.txt file, so I have kept size of table to be nearest prime number i.e., 58111.

### In Chianing it worked as whenever there's a collision, chaining is happening.

### In Linear Probing also it worked as whenever there's a collision, we are finding next available empty slot as we know the unique elements are less than size here, it worked

### 58111 didn't worked for Double Hashing, as the double hasing is going to infinte loop i.e., its only giving values which already are full which is expected as 58111 size and we cant expect hash function to give 58110 unique values but as soon as I increased the size to 158111 i.e, I increased the size by 1 lakh, it was able to generate 58110 unique keys and the problem solved.

## **Pros and Cons**

### **Chaining**

#### Pros
Handle high number of keys efficiently as long as load factor is not too high

#### Cons
When load factor becomes high, linked list can become too big to search

### **Linear Probing**

#### Pros
Easy to implement

#### Cons
Deleting makes the index not usable, so space is getting wasted

### **Double Hashing**

#### Pros
Better Distribution of Keys

#### Cons
Complicated to implement and if size is not enough it might end up in infintie loop

